In [1]:
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer

In [3]:
x1 = pd.ExcelFile('../data/PCA_data.xlsx')
df = x1.parse('new')#,header=None)
df2 = x1.parse('XGBoost')

print(df.head())
print(df.shape)
print(df2.head())

   Amplitude  Duration  Accumulated dose         N        DGR
0       1000       168              1000  2.609411  10.090759
1       1000       168              1000  2.979771   5.136975
2       1000       168              1000  2.865221   6.164093
3       2000         4              2000  1.999982 -15.801756
4       2000         4              2000  2.760342 -14.282713
(42, 5)
   Amplitude Duration Accumulated dose         N        DGR
0       1000     Long           <=1000  2.609411  10.090759
1       1000     Long           <=1000  2.979771   5.136975
2       1000     Long           <=1000  2.865221   6.164093
3       2000    Short           <=2000  1.999982 -15.801756
4       2000    Short           <=2000  2.760342 -14.282713


In [4]:
df.columns = df.columns.str.replace(' ','_')

In [5]:
df.head()

,Amplitude,Duration,Accumulated_dose,N,DGR
0,1000,168,1000,2.609411,10.090759
1,1000,168,1000,2.979771,5.136975
2,1000,168,1000,2.865221,6.164093
3,2000,4,2000,1.999982,-15.801756
4,2000,4,2000,2.760342,-14.282713


In [6]:
df.dtypes

Amplitude             int64
Duration              int64
Accumulated_dose      int64
N                   float64
DGR                 float64
dtype: object

In [7]:
df['N'].unique()

array([2.60941124, 2.97977114, 2.86522079, 1.99998224, 2.76034188,
       2.02693844, 2.17550278, 2.12057209, 1.13963091, 2.87504005,
       3.13118458, 2.93488932, 3.76830077, 4.23490095, 3.91720724,
       2.74445581, 3.35874772, 2.40742111, 2.3934567 , 2.25557375,
       2.75702429, 2.57193875, 2.94725585, 2.03067064, 2.21811104,
       2.73235106, 2.18040442, 2.63888049, 2.36608624,        nan,
       3.49629664, 1.91922283, 3.15910721, 1.45922875, 1.83478403,
       2.3710568 , 3.31671357, 2.26910806, 3.03569031, 1.61825013,
       1.79612362, 1.91841853])

In [8]:
len(df.loc[(np.isnan(df.N) == True)])

1

In [9]:
df = df.loc[(np.isnan(df.N) != True)]

In [ ]:
len(df)

In [ ]:
X = df.drop('N',axis = 1).copy()
X = X.drop('DGR',axis = 1).copy()
X = X.drop('Amplitude',axis = 1).copy()
X = X.drop('Accumulated_dose',axis = 1).copy()

X.head()

In [ ]:
y1 = df['DGR']
y2 = df['N']

print(y1.head())
print(y2.head())

In [ ]:
#pd.get_dummies(X,columns=['Amplitude','Duration','Accumulated_dose']).head()
pd.get_dummies(X,columns=['Duration']).head()

In [ ]:
#X_encoded = pd.get_dummies(X,columns=['Amplitude','Duration','Accumulated_dose'])
X_encoded = pd.get_dummies(X,columns=['Duration'])

X_encoded.head()

In [ ]:
y2.unique()

In [ ]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X_encoded,y1,test_size=0.25)#,stratify = yes)
X2_train, X2_test, y2_train, y2_test = train_test_split(X_encoded,y2,test_size=0.25)#,stratify = yes)

X_train, X_test, y1_train, y1_test = train_test_split(X_encoded,y1,test_size=0.5),stratify = y1)


X_train, X_test, y2_train, y2_test = train_test_split(X_encoded,y2,test_size=0.5)#,stratify = y2)


In [ ]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X,y1,test_size=0.2)#,stratify = yes)
X2_train, X2_test, y2_train, y2_test = train_test_split(X,y2,test_size=0.2)#,stratify = yes)

In [ ]:
X

In [ ]:
# param optimization 

# Round 1
param_grid = {
    'learning_rate': [0.1,0.05,0.01],
    'gamma': [0,0.25,1.0],
    'reg_lambda': [0,1.0,10.0],
    'scale_pos_rate': [1,3,5]
}



In [ ]:
optimal_params = GridSearchCV(
    estimator=xgb.XGBRegressor(objective='reg:squarederror'),
    param_grid = param_grid,
    scoring = 'roc_auc',
    verbose = 0,
    n_jobs = 10,
    cv = 3
                              )

optimal_params.fit(X1_train,
                  y1_train,
                  eval_metric='auc')
#                  eval_set=[(X1_test,y1_test)])
#                  verbose=False)

print(optimal_params.best_params_)

In [ ]:
xgbr = xgb.XGBRegressor(objective='reg:squarederror',verbosity=0) 
print(xgbr)

In [ ]:
xgbr = xgb.XGBRegressor(objective='reg:squarederror',n_estimators=10000,max_depth=1,
                        learning_rate=0.001,verbosity=0)#,booster='gbtree',scale_pos_weight=1) 

xgbr.fit(X2_train,
        y2_train)#,#,
        #verbose = True,
        #eval_metric = 'aucpr')#,
        #eval_set=[(X1_test,y1_test)])
 #       )

xgb.to_graphviz(xgbr,num_trees = 99, size="10,10")#,condition_node_params=)

In [ ]:
xgbr = xgb.XGBRegressor(objective='reg:linear',n_estimators=10000,max_depth=1,
                        learning_rate=0.001,verbosity=0)#,booster='gbtree',scale_pos_weight=1) 

xgbr.fit(X2_train,
        y2_train)#,#,
        #verbose = True,
        #eval_metric = 'aucpr')#,
        #eval_set=[(X1_test,y1_test)])
 #       )

xgb.to_graphviz(xgbr,num_trees = 99, size="10,10")#,condition_node_params=)

In [ ]:
y1.mean()

In [ ]:
plot_confusion_matrix(xgbr,X1_test,y1_test,values_format='d')

In [ ]:
y1

In [ ]:
xgb.to_graphviz(xgbr,num_trees = 0, size="10,10")#,condition_node_params=)

In [ ]:
len(y1_train)

In [ ]:
y1_test

In [ ]:
X

In [ ]:
xgbr = xgb.XGBRegressor(verbosity=0) 
print(xgbr)
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [ ]:
X_train, X_test, y1_train, y1_test = train_test_split(X_encoded,y1,test_size=0.25,stratify=y1)

In [ ]:
X_train, X_test, y1_train, y1_test = train_test_split(X,y1,stratify=y1)
X_train, X_test, y2_train, y2_test = train_test_split(X,y2,test_size=0.15,stratify='N')

In [ ]:
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
# Prepare the data data
#boston = datasets.load_boston()
#X = boston.data
#y = boston.target

#To keep the size of the tree small, I set max_depth = 3.

# Fit the regressor, set max_depth = 3
regr = DecisionTreeRegressor(max_depth=4, random_state=1234)
model = regr.fit(X_train, y1_train)
text_representation = tree.export_text(regr)
print(text_representation)

#' Amplitude','Duration','Accumulated_dose'

In [ ]:
import graphviz
from sklearn.datasets import load_iris
iris = load_iris()

# Model (can also use single decision tree)
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=10)

# Train
model.fit(iris.data, iris.target)
# Extract single tree
estimator = model.estimators_[5]

from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = iris.feature_names,
                class_names = iris.target_names,
                rounded = True, proportion = False, 
                precision = 2, filled = True)

# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
#Image(filename = 'tree.png')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=10)
# Train
model.fit(X,y1)#iris.data, iris.target)
# Extract single tree
estimator = model.estimators_[5]

In [ ]:
from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(#estimator_limited, 
                estimator,
                out_file='tree.dot', 
                feature_names = X.columns,
                class_names = iris.target_names,
                rounded = True, proportion = False, 
                precision = 2, filled = True)

In [ ]:
pip install graphviz

In [ ]:
# Convert to png
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

In [ ]:
X.columns

In [ ]:
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(regr, feature_names=['Amplitude', 'Duration', 'Accumulated_dose'])#, filled=True)

In [ ]:
y1

In [ ]:
with open("decistion_tree.log", "w") as fout:
    fout.write(text_representation)

In [ ]:
y1

y1

In [ ]:
from dtreeviz.trees import dtreeviz # remember to load the package

viz = dtreeviz(regr, X, y1)#,
                #target_name="target")#,
                #feature_names=X.columns)#,
                #class_names=list(['DGR']))

viz

In [ ]:
X.columns

In [ ]:
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(regr, 
                   feature_names=X.columns)#,  
                   #class_names=['DGR'])#,
                   #filled=True)

In [ ]:
xgbr = xgb.XGBRegressor(verbosity=0) 
print(xgbr)
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [ ]:
clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic', missing = None, seed = 42)
clf_xgb.fit(X_train,
           y1_train,
           verbose = True,
           eval_metric = 'aucpr',
           eval_set = [(X_test,y1_test)])

In [ ]:
def fit(self, X, y):
    """load the data in, initiate the models"""
    self.X = X
    self.y = y
    self.opt_XGBoost_reg = xgb.XGBRegressor(**self.opt_xgb_params)
    self.opt_forest_reg = RandomForestRegressor(**self.opt_rf_params)
    self.opt_svm_reg = SVR(**self.opt_svm_params)
    """ fit the models """
    self.opt_XGBoost_reg.fit(self.X ,self.y)
    self.opt_forest_reg.fit(self.X ,self.y)
    self.opt_svm_reg.fit(self.X ,self.y) 

In [ ]:
fit(X,X,y1)

In [ ]:
xtrain, xtest, ytrain, ytest=train_test_split(x, y, test_size=0.15)

In [ ]:
model = xgb.XGBRegressor()
model.fit(X_train, y1_train)
print(); print(model)

In [ ]:
expected_y  = y1_test
predicted_y = model.predict(X_test)

In [ ]:
print(metrics.r2_score(expected_y, predicted_y))
print(metrics.mean_squared_log_error(expected_y, predicted_y))

plt.figure(figsize=(10,10))
sns.regplot(expected_y, predicted_y, fit_reg=True, scatter_kws={"s": 100})

In [ ]:
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("ggplot")

import xgboost as xgb

In [ ]:
xl = pd.ExcelFile('../data/PCA_data.xlsx')
df = xl.parse('new')#,header=None)

print(df.head())
print(df.shape)

In [ ]:
df.columns = df.columns.str.replace(' ','_')

In [ ]:
df = df.loc[(np.isnan(df.N) != True)]

In [ ]:
X = df.drop('N',axis = 1).copy()
X = X.drop('DGR',axis = 1).copy()

X.head()

In [ ]:
y1 = df['DGR']
y2 = df['N']

print(y1.head())
print(y2.head())

In [ ]:
X_encoded = pd.get_dummies(X,columns=['Amplitude','Duration','Accumulated_dose'])
X_encoded.head()

In [ ]:
X_train, X_test, y1_train, y1_test = train_test_split(X,y1,test_size=0.15)#,stratify = yes)
X_train, X_test, y2_train, y2_test = train_test_split(X,y2,test_size=0.15)#,stratify = yes)

In [ ]:
expected_y1  = y1_test
predicted_y1 = model.predict(X_test)

In [ ]:
expected_y2  = y2_test
predicted_y2 = model.predict(X_test)

In [ ]:
model = xgb.XGBRegressor()
model.fit(X_train, y1_train)
print(); print(model)

In [ ]:
model = xgb.XGBRegressor()
model.fit(X_train, y2_train)
print(); print(model)

In [ ]:
expected_y1  = y1_test
predicted_y1 = model.predict(X_test)

In [ ]:
expected_y2  = y2_test
predicted_y2 = model.predict(X_test)

In [ ]:
print(metrics.r2_score(expected_y1, predicted_y1))
print(metrics.mean_squared_log_error(expected_y1, predicted_y1))

plt.figure(figsize=(10,10))
sns.regplot(expected_y1, predicted_y1, fit_reg=True, scatter_kws={"s": 100})

In [ ]:
print(metrics.r2_score(expected_y2, predicted_y2))
print(metrics.mean_squared_log_error(expected_y2, predicted_y2))

plt.figure(figsize=(10,10))
sns.regplot(expected_y2, predicted_y2, fit_reg=True, scatter_kws={"s": 100})